In [1]:
import cv2
import math
import numpy as np 
import pyautogui

In [7]:
# Opening camera 

vid = cv2.VideoCapture(0)

while vid.isOpened():
    ret, frame = vid.read()
    # cv2.imshow('frame', frame)
    cv2.rectangle(frame, (100, 100), (500,500), (0,0,255), 2)
    crop_image = frame[100:500, 100:500]
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break

    # Blurring to smoothen edges 
    blur = cv2.GaussianBlur(crop_image, (3,3), 0)

    # Converting to HSV
    hsv = cv2.cvtColor(blur, cv2.COLOR_BGR2HSV)

    # Creating a mask - skin colors in white and rest black 
    mask = cv2.inRange(hsv, np.array([2,0,0]), np.array([30,255,255]))

    # Morphological operations
    kernel = np.ones((5,5))
    dilation = cv2.dilate(mask, kernel, iterations=1)
    erosion = cv2.erode(dilation, kernel, iterations=1)

    blur2 = cv2.GaussianBlur(erosion, (3,3), 0)
    ret, thresh = cv2.threshold(blur2, 127, 255, cv2.THRESH_BINARY)

    # Finding contours 
    contours, heirarchy = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    try:
        contour = max(contours, key=lambda x: cv2.contourArea(x))

        x, y, w, h = cv2.boundingRect(contour)
        cv2.rectangle(crop_image, (x, y), (x+w, y+h), (0,0,255), 0)

        hull = cv2.convexHull(contour)

        drawing = np.zeros(crop_image.shape, np.uint8)
        cv2.drawContours(drawing, [contour], -1, (0,255,0), 0)
        cv2.drawContours(drawing, [hull], -1, (0,0,255), 0)

        hull = cv2.convexHull(contour, returnPoints = False)
        defects = cv2.convexityDefects(contour, hull)

        count_defects = 0

        for i in range(defects.shape[0]):
            s, e, f, d = defects[i,0]
            start = tuple(contour[s][0])
            end = tuple(contour[e][0])
            far = tuple(contour[f])[0]

            a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
            b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
            c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
            angle = (math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c)) * 180) / 3.14

            if angle <= 90:
                count_defects += 1
                cv2.circle(crop_image, far, 1, [0, 0, 255], -1)

            cv2.line(crop_image, start, end, [0,255,0], 2)

        if count_defects >= 4:
            pyautogui.press('space')
            cv2.putText(frame, "Jump", (115, 80), cv2.FONT_HERSHEY_SIMPLEX, 2, 2, 2)

    except: 
        pass

    cv2.imshow("Gesture", frame)

    if cv2.waitKey(1) == ord('q'):
        break

vid.release()
cv2.destroyAllWindows()

[ WARN:0@398.585] global /private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_562_cazh1h/croots/recipe/opencv-suite_1664548333142/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
